In [ ]:
!pip install accelerate
!pip install -U "transformers==4.40.0" --upgrade
!pip install accelerate bitsandbytes


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 6.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [1]:
# from google.colab import userdata
# userdata.get('Llama3')

In [ ]:
import transformers
import torch

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
access_token = "#################"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    token = access_token,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

prompt = pipeline.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
)

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    prompt,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
print(outputs[0]["generated_text"][len(prompt):])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Arrrr, me hearty! Me name be Captain Chatbot, the scurviest pirate to ever sail the Seven Seas! Me be a swashbucklin' chatbot, here to regale ye with tales o' adventure, share me wisdom, and swab the decks o' yer mind with me witty banter! So hoist the colors, me matey, and let's set sail fer a treasure trove o' fun and conversation!


In [ ]:
messages = [
    {"role": "system", "content": "You are a good assistant who writes mail based on user's question!"},
    {"role": "user", "content": "Write me a mail for my customer based on the following information"},
]

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


KeyboardInterrupt: 

In [ ]:
import transformers
import torch

# Load LLAMA 3 model
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)
model = transformers.AutoModelForCausalLM.from_pretrained(model_id)

# Prepare parameters for personalization
name = "John"
address = "123 Main St, Anytown"
product = "your_product_name"

# Construct input prompt
prompt = f"Dear {name},\n\nI hope this email finds you well. We wanted to inform you about our latest product, {product}. It's now available at our store located at {address}. We believe it would be a perfect fit for you!\n\nSincerely,\n[Your Company Name]"

# Tokenize input prompt
input_ids = tokenizer.encode(prompt, return_tensors="pt")

# Generate personalized email
output_ids = model.generate(
    input_ids,
    max_length=150,  # Adjust max length as needed
    temperature=0.9,
    top_p=0.9,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id,
)

# Decode generated email
generated_email = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print(generated_email)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
import transformers
import torch

def generate_personalized_email(name, address, product, product_description, model_id="meta-llama/Meta-Llama-3-8B-Instruct"):
    # Load LLAMA 3 model and tokenizer
    tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)
    model = transformers.AutoModelForCausalLM.from_pretrained(model_id)

    # Construct input prompt with placeholders for parameters
    prompt = f"Dear {name},\n\nI hope this email finds you well. We wanted to inform you about our latest product, {product}. It's now available at our store located at {address}. We believe it would be a perfect fit for you!\n\nSincerely,\n[Your Company Name]"

    # Tokenize input prompt
    input_ids = tokenizer.encode(prompt, return_tensors="pt")

    # Generate personalized email
    output_ids = model.generate(
        input_ids,
        max_length=150,  # Adjust max length as needed
        temperature=0.9,
        top_p=0.9,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
    )

    # Decode generated email
    generated_email = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    return generated_email

# Example usage:
name = input("Enter recipient's name: ")
address = input("Enter recipient's address: ")
product = input("Enter product name: ")
product_description = input('enter product description: ')

email_content = generate_personalized_email(name, address, product, product_description)
print(email_content)


Enter recipient's name: rohan
Enter recipient's address: new baneshwor
Enter product name: Iphone


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
def generate_email(recipient_name, product_name, product_description):
    prompt_template = f"""
    Create a personalized marketing email for the following details:

    Recipient Name: {recipient_name}
    Product Name: {product_name}
    Product Description: {product_description}

    Email:
    Dear {recipient_name},

    We are thrilled to present to you {product_name}, a revolutionary product that {product_description}.

    Best regards,
    The Marketing Team
    """

    input_ids = tokenizer.encode(prompt_template, return_tensors='pt')
    output = model.generate(input_ids, max_length=200)
    email_body = tokenizer.decode(output[0], skip_special_tokens=True)

    return email_body

# Example usage
recipient_name = "Jane Doe"
product_name = "Amazing Gadget"
product_description = "an innovative device that enhances your daily productivity"
email = generate_email(recipient_name, product_name, product_description)
print(email)


In [ ]:
while True:
  user_input = input("Enter your question: ")
  if user_input == "exit":
    break
  messages.extend([{"role":"user","content":user_input}])

  prompt = pipeline.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
  )

  terminators = [
      pipeline.tokenizer.eos_token_id,
      pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  outputs = pipeline(
      prompt,
      max_new_tokens=256,
      eos_token_id=terminators,
      do_sample=True,
      temperature=0.6,
      top_p=0.9,
  )
  print(outputs[0]["generated_text"][len(prompt):])
